<a href="https://colab.research.google.com/github/chongzicbo/keras_tutorial/blob/master/keras%E5%87%BD%E6%95%B0%E5%BC%8F(Functional)%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. 第个模型：全连接网络
首先弄清楚几个概念：
* 层对象接受张量为参数，返回一个张量
* 输入是张量，输出也是张量的一个框架就是一个模型，通过Model类定义。
* 这样的模型可以被像Sequential一样被训练

In [0]:
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras import datasets
import tensorflow as tf
import numpy as np

In [0]:
inputs=Input(shape=(784,)) #Input会返回一个张量
x=Dense(64,activation='relu')(inputs) #Input返回的向量作为Dense层的输入，Dense层也返回一个向量
x=Dense(64,activation='relu')(x)
predictions=Dense(10,activation='softmax')(x)

#使用Model类创建一个模型
model=Model(inputs=inputs,outputs=predictions)
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

创建训练数据集

In [0]:
(x_train,y_train),(x_test,y_test)=datasets.mnist.load_data()

In [0]:
train_data=(x_train.reshape((x_train.shape[0],x_train.shape[1]*x_train.shape[2])),y_train)
test_data=(x_test.reshape((x_test.shape[0],x_test.shape[1]*x_test.shape[2])),y_test)

开始训练模型

In [17]:
model.fit(x=train_data[0],y=train_data[1],batch_size=256,epochs=10,validation_data=test_data)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s 24us/sample - loss: 3.4795 - acc: 0.8057 - val_loss: 0.9358 - val_acc: 0.8639
Epoch 2/10
60000/60000 [==============================] - 1s 17us/sample - loss: 0.5623 - acc: 0.9007 - val_loss: 0.4798 - val_acc: 0.9076
Epoch 3/10
60000/60000 [==============================] - 1s 19us/sample - loss: 0.3479 - acc: 0.9235 - val_loss: 0.3225 - val_acc: 0.9318
Epoch 4/10
60000/60000 [==============================] - 1s 17us/sample - loss: 0.2595 - acc: 0.9402 - val_loss: 0.4084 - val_acc: 0.9186
Epoch 5/10
60000/60000 [==============================] - 1s 17us/sample - loss: 0.2130 - acc: 0.9476 - val_loss: 0.3250 - val_acc: 0.9346
Epoch 6/10
60000/60000 [==============================] - 1s 17us/sample - loss: 0.1881 - acc: 0.9546 - val_loss: 0.3079 - val_acc: 0.9361
Epoch 7/10
60000/60000 [==============================] - 1s 17us/sample - loss: 0.1623 - acc: 0.9595 - val_loss: 0.

##2. 所有的模型都是可调用的，就像层一样
利用函数式模型的接口，我们可以很容易的重用已经训练好的模型：你可以把模型当作一个层一样，通过提供一个tensor来调用它。注意当你调用一个模型时，你不仅仅重用了它的结构，也重用了它的权重。

In [0]:
x=Input(shape=(784,))
y=model(x)

In [0]:
from tensorflow.keras.layers import TimeDistributed

#时间步长为20的输入向量，每个时间步的向量维度为784
input_sequences=Input(shape=(20,784))
#将之前的模型应用到输入序列的每一个时间步，最终的输出shape为[batch_size,20,10]
processed_sequences=TimeDistributed(model)(input_sequences)

In [22]:
processed_sequences.shape

TensorShape([Dimension(None), Dimension(20), Dimension(10)])

## 3.多输入和多输出模型

使用函数式模型的一个典型场景是搭建多输入、多输出的模型。

考虑这样一个模型。我们希望预测Twitter上一条新闻会被转发和点赞多少次。模型的主要输入是新闻本身，也就是一个词语的序列。但我们还可以拥有额外的输入，如新闻发布的日期等。这个模型的损失函数将由两部分组成，辅助的损失函数评估仅仅基于新闻本身做出预测的情况，主损失函数评估基于新闻和额外信息的预测的情况，即使来自主损失函数的梯度发生弥散，来自辅助损失函数的信息也能够训练Embeddding和LSTM层。在模型中早点使用主要的损失函数是对于深度网络的一个良好的正则方法。总而言之，该模型框图如下：
<center><img src="https://keras-cn.readthedocs.io/en/latest/images/multi-input-multi-output-graph.png" width="300"/></center>

让我们用函数式模型来实现这个框图

主要的输入接收新闻本身，即一个整数的序列（每个整数编码了一个词）。这些整数位于1到10，000之间（即我们的字典有10，000个词）。这个序列有100个单词。



In [0]:
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense
from tensorflow.keras.models import Model

In [0]:
#输入序列长度为100，序号再=在1-10000之间
main_input=Input(shape=(100,),dtype='int32',name='main_input')
#经过Embedding后，输出shape为(batch_size,input_length,output_dim)
x=Embedding(output_dim=512,input_dim=10000,input_length=100)(main_input)
#使用lstm进行编码，输出32个神经元
lstm_out=LSTM(32)(x)

In [33]:
lstm_out.shape

TensorShape([Dimension(None), Dimension(32)])

然后，我们插入一个额外的损失，使得即使在主损失很高的情况下，LSTM和Embedding层也可以平滑的训练。

In [0]:
auxiliary_output=Dense(1,activation='sigmoid',name='aux_output')(lstm_out)

再然后，我们将LSTM与额外的输入数据串联起来组成输入，送入模型中：

In [35]:
auxiliary_input=Input(shape=(5,),name='aux_input')
x=keras.layers.concatenate([lstm_out,auxiliary_input])
x.shape

TensorShape([Dimension(None), Dimension(37)])

In [0]:
x=Dense(64,activation='relu')(x)
x=Dense(64,activation='relu')(x)
x=Dense(64,activation='relu')(x)
main_output=Dense(1,activation='sigmoid',name='main_output')(x)

最后，我们定义整个2输入，2输出的模型：

In [0]:
model=Model(inputs=[main_input,auxiliary_input],outputs=[main_output,auxiliary_output])

创建假的数据集进行训练

In [0]:
x_main=tf.reshape(tf.range(0,10000),shape=(100,100))
y_main=tf.concat([tf.zeros(shape=(50)),tf.ones(shape=(50))],axis=0)

x_auxiliary=tf.reshape(tf.range(0,500),shape=(100,5))
y_auxiliary=tf.concat([tf.zeros(shape=(50)),tf.ones(shape=(50))],axis=0)

模型定义完毕，下一步编译模型。我们给额外的损失赋0.2的权重。我们可以通过关键字参数loss_weights或loss来为不同的输出设置不同的损失函数或权值。这两个参数均可为Python的列表或字典。这里我们给loss传递单个损失函数，这个损失函数会被应用于所有输出上。

In [51]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',loss_weights=[1.,0.2])
model.fit(x=[x_main,x_auxiliary],y=[y_main,y_auxiliary],epochs=50,batch_size=10,steps_per_epoch=10)

Train on 10 samples
Epoch 1/50
10/10 [==============================] - 4s 435ms/step - loss: 2.3195 - main_output_loss: 2.2065 - aux_output_loss: 0.5646
Epoch 2/50
10/10 [==============================] - 3s 342ms/step - loss: 0.8313 - main_output_loss: 0.7815 - aux_output_loss: 0.2487
Epoch 3/50
10/10 [==============================] - 3s 345ms/step - loss: 0.6167 - main_output_loss: 0.5863 - aux_output_loss: 0.1518
Epoch 4/50
10/10 [==============================] - 3s 337ms/step - loss: 0.8276 - main_output_loss: 0.8082 - aux_output_loss: 0.0968
Epoch 5/50
10/10 [==============================] - 3s 349ms/step - loss: 0.0485 - main_output_loss: 0.0347 - aux_output_loss: 0.0688
Epoch 6/50
10/10 [==============================] - 3s 343ms/step - loss: 0.0132 - main_output_loss: 0.0033 - aux_output_loss: 0.0495
Epoch 7/50
10/10 [==============================] - 3s 334ms/step - loss: 0.0081 - main_output_loss: 0.0013 - aux_output_loss: 0.0343
Epoch 8/50
10/10 [========================

In [47]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 512)     5120000     main_input[0][0]                 
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           69760       embedding_1[0][0]                
__________________________________________________________________________________________________
aux_input (InputLayer)          [(None, 5)]          0                                            
____________________________________________________________________________________________

##4.共享层
另一个使用函数式模型的场合是使用共享层的时候。

考虑微博数据，我们希望建立模型来判别两条微博是否是来自同一个用户，这个需求同样可以用来判断一个用户的两条微博的相似性。

一种实现方式是，我们建立一个模型，它分别将两条微博的数据映射到两个特征向量上，然后将特征向量串联并加一个logistic回归层，输出它们来自同一个用户的概率。这种模型的训练数据是一对对的微博。

因为这个问题是对称的，所以处理第一条微博的模型当然也能重用于处理第二条微博。所以这里我们使用一个共享的LSTM层来进行映射。

首先，我们将微博的数据转为（140，256）的矩阵，即每条微博有140个字符，每个单词的特征由一个256维的词向量表示，向量的每个元素为1表示某个字符出现，为0表示不出现，这是一个one-hot编码。

之所以是（140，256）是因为一条微博最多有140个字符，而扩展的ASCII码表编码了常见的256个字符。原文中此处为Tweet，所以对外国人而言这是合理的。如果考虑中文字符，那一个单词的词向量就不止256了。

In [0]:
tweet_a=Input(shape=(140,256))
tweet_b=Input(shape=(140,256))

若要对不同的输入共享同一层，就初始化该层一次，然后多次调用它

In [0]:
shared_lstm=LSTM(64)
encoded_a=shared_lstm(tweet_a)
encoded_b=shared_lstm(tweet_b)
merged_vector=keras.layers.concatenate([encoded_a,encoded_b],axis=-1)
predictions=Dense(1,activation='sigmoid')(merged_vector)

In [0]:
model=Model(inputs=[tweet_a,tweet_b],outputs=predictions)
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
data_a=tf.reshape(tf.range(start=0,limit=100*140*256),shape=(100,140,256))
data_b=tf.reshape(tf.range(start=0,limit=100*140*256),shape=(100,140,256))
labels=tf.concat([tf.zeros(shape=(50,)),tf.ones(shape=(50,))],axis=0)

In [57]:
data_a.shape,data_b.shape,labels.shape

(TensorShape([Dimension(100), Dimension(140), Dimension(256)]),
 TensorShape([Dimension(100), Dimension(140), Dimension(256)]),
 TensorShape([Dimension(100)]))

In [0]:
model.fit([data_a,data_b],labels,epochs=10,steps_per_epoch=10)

##5. 层“节点”的概念
无论何时，当你在某个输入上调用层时，你就创建了一个新的张量（即该层的输出），同时你也在为这个层增加一个“（计算）节点”。这个节点将输入张量映射为输出张量。当你多次调用该层时，这个层就有了多个节点，其下标分别为0，1，2...

在上一版本的Keras中，你可以通过layer.get_output()方法来获得层的输出张量，或者通过layer.output_shape获得其输出张量的shape。这个版本的Keras你仍然可以这么做（除了layer.get_output()被output替换）。但如果一个层与多个输入相连，会出现什么情况呢？

如果层只与一个输入相连，那没有任何困惑的地方。.output将会返回该层唯一的输出

In [0]:
a=Input(shape=(140,256))
lstm=LSTM(32)
encoded_a=lstm(a)
assert lstm.output==encoded_a

但当层与多个输入相连时，会出现问题

In [68]:
a=Input(shape=(140,256))
b=Input(shape=(140,256))
lstm=LSTM(32)
encoded_a=lstm(a)
encoded_b=lstm(b)
lstm.output #默认输出第一个节点：encoded_a

<tf.Tensor 'lstm_9/strided_slice_7:0' shape=(?, 32) dtype=float32>

In [65]:
lstm.get_output_at(0)

<tf.Tensor 'lstm_7/strided_slice_7:0' shape=(?, 32) dtype=float32>

In [0]:
assert lstm.output==encoded_a

In [70]:
assert lstm.output==encoded_b

AssertionError: ignored